# Co-Vid19 Data Science Team Project
## Jason Lubrano, Andrea Chamorro
### University of Colorado Boulder, Department of Computer Science

In [1]:
# imports
import numpy as np 
from scipy import stats
import statistics
import statsmodels.api as sm 
import pandas as pd
import matplotlib.pylab as plt 
%matplotlib inline

In [25]:
# first off, watermelon oreos are the best flavor no matter what the statistics say
dfCOVID = pd.read_csv("latestdata.csv")

dfCOVID.dropna(subset=['age', 'sex', 'city', 'country'], inplace = True)

covidColors = [
    '#68b4e3', # blue
    '#ccd663', # yellow-green
    '#262626', # black-ish
    '#e85f35', # orange
    '#7bc74c', # green
]

dfCOVID.drop_duplicates(inplace=True)

In [26]:
dfCOVID.head()

,ID,age,sex,city,province,country,wuhan(0)_not_wuhan(1),latitude,longitude,geo_resolution,...,date_death_or_discharge,notes_for_discussion,location,admin3,admin2,admin1,country_new,admin_id,data_moderator_initials,travel_history_binary
1,000-1-,50-59,male,Snohomish County,Washington,United States,1.0,48.04818,-121.6960,admin2,...,NaN,NaN,NaN,NaN,Snohomish County,Washington,United States,2988,NaN,NaN
7,000-1-,60-69,female,Snohomish County,Washington,United States,1.0,48.04818,-121.6960,admin2,...,NaN,NaN,NaN,NaN,Snohomish County,Washington,United States,2988,NaN,NaN
8,000-1-,50-59,male,Snohomish County,Washington,United States,1.0,48.04818,-121.6960,admin2,...,NaN,NaN,NaN,NaN,Snohomish County,Washington,United States,2988,NaN,NaN
9,000-1-,30-39,female,Snohomish County,Washington,United States,1.0,48.04818,-121.6960,admin2,...,NaN,NaN,NaN,NaN,Snohomish County,Washington,United States,2988,NaN,NaN
11,000-1-1,30,male,"Chaohu City, Hefei City",Anhui,China,1.0,31.64696,117.7166,admin3,...,NaN,NaN,NaN,Chaohu City,Hefei City,Anhui,China,340181,NaN,NaN


In [37]:
dfCOVID.groupby(level='ID')

ValueError: level name ID is not the name of the index